# 自前のネットワークを組んで学習させてみる

In [16]:
import pandas as pd
import numpy as np
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
plt.rcParams['font.family'] = 'Meiryo'

In [17]:
train_fn = r'./dataset_cur_train.csv'
valid_fn = r'./dataset_cur_valid.csv'

In [18]:
df_train = pd.read_csv(train_fn,index_col=0)
df_train['red_diff'] = df_train['red_diff'].astype(np.float16)
df_train['remain_ends'] = df_train['remain_ends'].astype(np.float16)
df_train['last_stone_is_red'] = df_train['last_stone_is_red'].astype(np.float16)
df_train['red_postion'] = df_train['red_postion'].astype(np.float16)
display(df_train.dtypes)
display(df_train.head(3))

red_diff             float16
remain_ends          float16
last_stone_is_red    float16
red_postion          float16
filepath              object
dtype: object

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
3,-2.0,5.0,1.0,-2.0,./dataset_o\WWCC2022_ResultsBook\geme536end4.png
0,2.0,8.0,1.0,0.0,./dataset_o\WMCC2023_ResultsBook\geme271end1.png
8,-3.0,0.0,0.0,4.0,./dataset_o\WMCC2023_ResultsBook\geme346end9.png


## red_diff が目的、fn,remain_ends,last_stone_is_red,red_postion が説明変数
説明変数を標準化する

In [19]:
# 標準化
stdsc = StandardScaler()
##学習時の標準化したパラメータは、評価、本番時におなじ重みで標準化する処理が必要
x_train_df = df_train.copy().drop(['filepath','red_diff'],axis=1)
x_train_std = stdsc.fit_transform(x_train_df)
display( x_train_std[:3] )
## DataFrameの値を入れ替え
qcl = df_train.columns.to_list()
qcl.remove('filepath')
qcl.remove('red_diff')
print(qcl)
df_train[qcl] = x_train_std
pickle.dump(stdsc, open("stdsc_02240209.pkl", "wb"))
display(stdsc.n_features_in_, stdsc.mean_ , stdsc.var_) 
df_train.head(5)

array([[ 0.3816,  0.9927, -0.846 ],
       [ 1.538 ,  0.9927,  0.0291],
       [-1.546 , -1.007 ,  1.78  ]], dtype=float16)

['remain_ends', 'last_stone_is_red', 'red_postion']


3

array([ 4.00979228,  0.50356083, -0.06646884])

array([6.73195159, 0.24998732, 5.21872729])

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
3,-2.0,0.381592,0.992676,-0.846191,./dataset_o\WWCC2022_ResultsBook\geme536end4.png
0,2.0,1.538086,0.992676,0.029099,./dataset_o\WMCC2023_ResultsBook\geme271end1.png
8,-3.0,-1.545898,-1.006836,1.780273,./dataset_o\WMCC2023_ResultsBook\geme346end9.png
2,0.0,0.767090,0.992676,0.466797,./dataset_o\WMCC2023_ResultsBook\geme68end3.png
1,-1.0,1.152344,-1.006836,0.029099,./dataset_o\WWCC2023_ResultsBook\geme158end2.png


### 20240209_samplescreeningで画像の平均 分散は計算ずみ
平均:[0.525406002998352, 0.5207120180130005, 0.5381810665130615]  分散:
[0.47722673416137695, 0.4751463830471039, 0.48789089918136597  ]

In [21]:
import torch
from torchvision import transforms
import torchvision


In [22]:
## 画像変換の定義
w,h = Image.open(df_train['filepath'].values[0]).size
## 正方形にするための差分
pad = (h-w)/2
display([w,h],pad)
target_size = 224
## train用
transform_train = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5254, 0.521, 0.538], std=[0.477, 0.475, 0.487])
])
# valid/test用
transform_test = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5254, 0.521, 0.538], std=[0.477, 0.475, 0.487])
])

[300, 540]

120.0

In [42]:
test_df.head(2)

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
6,-2.0,-0.774414,-1.006836,0.904785,./dataset_o\WWCC2022_ResultsBook\geme374end7.png
4,0.0,-0.003773,-1.006836,0.029099,./dataset_o\WMCC2021_ResultsBook\geme656end5.png


In [23]:
from torch.utils.data import DataLoader, Dataset

# 自前のデータセット定義

In [24]:
class ImgValueDataset(Dataset):
    def __init__(self, df, classcol , fncol , transform):
        
        ##self.label_list  = df[classcol].to_list()
        self.label_list  = pd.get_dummies(df[classcol]).values
        self.img_pathlist  = df[fncol].to_list()
        cols = df.columns.to_list()
        cols.remove(fncol)
        cols.remove(classcol)
        self.x_values = df[cols].values
        self.transform = transform

    def __len__(self):  
        return len( self.img_pathlist )
    
    def __getitem__(self, index):
        # 画像をPILとして読み込む
        #print(index)
        image = Image.open(self.img_pathlist[index])
        
        label = self.label_list[index]
                             
        if self.transform is not None:
            ##print('use transform')
            image = self.transform(image)
        return image, label 
    pass

In [26]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_train, test_size=0.2, stratify=df_train['red_diff'])
train_dataset = ImgValueDataset( train_df ,classcol='red_diff' , fncol='filepath',transform=transform_train)
test_dataset = ImgValueDataset( test_df ,classcol='red_diff' , fncol='filepath' ,transform=transform_test)

In [27]:
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


In [28]:
images, labels = next(iter(train_loader))
images.shape , labels

(torch.Size([4, 3, 224, 224]),
 tensor([[False, False, False, False,  True, False, False, False, False, False,
          False],
         [False, False, False, False, False, False,  True, False, False, False,
          False],
         [False, False, False, False, False, False, False, False,  True, False,
          False],
         [False, False, False, False, False, False, False,  True, False, False,
          False]]))

In [29]:
class_list = train_df['red_diff'].unique()
class_list , len(class_list)

(array([ 1., -1.,  2., -2.,  0.,  3., -4., -3., -5.,  4.,  5.],
       dtype=float16),
 11)

試しに汎用モデルを読む

In [30]:
os.environ["http_proxy"]

KeyError: 'http_proxy'

In [31]:
""" あきらめ
from torchvision import models 
import pprint
from torchvision.models import MobileNetV2
os.environ["http_proxy"] = "http://zl-hlb.ngk.co.jp:8080"
os.environ["https_proxy"] = "http://zl-hlb.ngk.co.jp:8080"
Model = models.mobilenet_v2(pretrained=True)
Model
"""

' あきらめ\nfrom torchvision import models \nimport pprint\nfrom torchvision.models import MobileNetV2\nos.environ["http_proxy"] = "http://zl-hlb.ngk.co.jp:8080"\nos.environ["https_proxy"] = "http://zl-hlb.ngk.co.jp:8080"\nModel = models.mobilenet_v2(pretrained=True)\nModel\n'

# ネットを組む  
https://qiita.com/poorko/items/c151ff4a827f114fe954

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
# CNNモデルの定義
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        
        self.fc1 = nn.Linear(24*106*106, 10240)
        self.fc2 = nn.Linear(10240, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, input):
      
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))   
        ###print(output.shape)
        output = output.view(-1, 24*106*106)
        output = F.relu(self.fc1(output))  # fc1->relu
        output = F.relu(self.fc2(output))  # fc2->relu
        output = self.fc3(output)
        
        return output

In [34]:
net = SimpleCNN(11)
print(net)

SimpleCNN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=269664, out_features=10240, bias=True)
  (fc2): Linear(in_features=10240, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=11, bias=True)
)


## 損失関数とオプティマイザ

In [35]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

## 訓練（ネットワークが通るか確認）

In [43]:
from torch.autograd import Variable
def saveModel():
    path = "./20240209.Model.pth"
    torch.save(model.state_dict(), path)
    
def testAccuracy(model,test_loader):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

## 学習可能か確認

In [45]:
num_epochs = 2
best_accuracy = 0.0
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,drop_last = True)
test_loader = DataLoader(test_df, batch_size=batch_size, shuffle=True, num_workers=0)

# Define your execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")
net.to(device)
min_loss = 999999999
for epoch in range(num_epochs):  # loop over the dataset multiple times
    train_loss = 0.0;val_loss = 0.0
    train_batches = 0;val_batches = 0
    running_loss = 0.0 
    net.train()  # 訓練モード
    for index_minbatch , (inputs, labels) in enumerate(train_loader, 0):
        print(f'min_batch: start:{index_minbatch}')
        inputs ,labels_float  = inputs.to(device) , labels.float().to(device)
        # 勾配のリセット
        optimizer.zero_grad()
        
        outputs = net(inputs)    # 順方向計算
        loss = criterion(outputs, labels_float)   # 損失の計算
        loss.backward()                     # 逆方向計算(勾配計算)
        optimizer.step()                    # パラメータの更新  

        # 損失関数の変化を１0ミニバッチごとに表示
        running_loss += loss.item()     # extract the loss value
        if index_minbatch % 10 == 9:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, index_minbatch + 1, running_loss / 10))
            # zero the loss
            running_loss = 0.0
            pass
    #########################################
    accuracy = testAccuracy(net,test_loader)
    print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
    # we want to save the model if the accuracy is the best
    if accuracy > best_accuracy:
        saveModel()
        best_accuracy = accuracy

print('Finished Training')

The model will be running on cuda:0 device
min_batch: start:0
min_batch: start:1
min_batch: start:2
min_batch: start:3
min_batch: start:4
min_batch: start:5
min_batch: start:6
min_batch: start:7
min_batch: start:8
min_batch: start:9
[1,    10] loss: 14.195
min_batch: start:10
min_batch: start:11
min_batch: start:12
min_batch: start:13
min_batch: start:14
min_batch: start:15
min_batch: start:16
min_batch: start:17
min_batch: start:18
min_batch: start:19
[1,    20] loss: 2.615
min_batch: start:20
min_batch: start:21
min_batch: start:22
min_batch: start:23
min_batch: start:24
min_batch: start:25
min_batch: start:26
min_batch: start:27
min_batch: start:28
min_batch: start:29
[1,    30] loss: 2.398
min_batch: start:30
min_batch: start:31
min_batch: start:32
min_batch: start:33
min_batch: start:34
min_batch: start:35
min_batch: start:36
min_batch: start:37
min_batch: start:38
min_batch: start:39
[1,    40] loss: 2.389
min_batch: start:40
min_batch: start:41


KeyError: 503

In [ ]:
outputs.dtype , labels_float.dtype ,labels.dtype

In [ ]:
outputs.shape , labels_float.shape

In [44]:
len(train_loader)

42

In [ ]:
labels_float